In [12]:
import os
import pandas as pd
from random import seed
from random import randrange
from math import sqrt
from csv import reader
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_csv(filename, header):
    dset = list()
    with open(filename, 'r') as file: 
        csv_reader = reader(file)
        for row in csv_reader:    
            if not row or not header:
                header = 1
                continue
            dset.append(row)
    return dset

def str_col_to_float(dset, col):
    for row in dset:
        row[col] = float(row[col].strip())
        
def str_col_to_int(dset, col):
    class_values = [row[col] for row in dset]
    uniq = set(class_values)
    lookup = dict()
    for i, val in enumerate(uniq):
        lookup[val] = i
    for row in dset:
        row[col] = lookup[row[col]]
    return lookup  


def cross_validation_split(dset, k):
    fold_size = int(len(dset)/k)
    dset_cpy = list(dset)
    dset_split = list() 
    for i in range(k):
        fold = list()
        while len(fold) < fold_size:
            fold.append(dset_cpy.pop(randrange(len(dset_cpy))))
        dset_split.append(fold)
    return dset_split


def test_split(col, val, dset):
    left, right = list(), list()
    for row in dset:
        if row[col] < val:
            left.append(row)
        else:
            right.append(row)
    return left, right


def accuracy_metric(actual, predicted):
    correct = 0.0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct/float(len(actual)) * 100.0


def eval_algorithm(dset, algorithm, k, *args):
    folds = cross_validation_split(dset, k)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_cpy = list(row)
            test_set.append(row_cpy)
            row_cpy[-1] = None 
        pred = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, pred)
        scores.append(accuracy)
    return scores


def gini_index(groups, class_values):
    size0 = float(len(groups[0]))
    size1 = float(len(groups[1]))
    gini = 0
    for group in groups:
        if len(group) == 0:
            continue
        gini_group = 0
        for class_val in class_values:
            proportion = [row[-1] for row in group].count(class_val) / float(len(group))
            gini_group += (proportion*(1.0 - proportion))
        gini += gini_group 
    return gini


def get_split(dset, n_features):  
    class_values = list(set([row[-1] for row in dset]))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    feat_sub = list()
    while len(feat_sub) < n_features: 
        index = randrange(len(dset[0]) - 1)
        if index in feat_sub:
            continue
        feat_sub.append(index)
    for feature in feat_sub:
        for row in dset:
            groups = test_split(feature, row[feature], dset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = feature, row[feature], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

def to_terminal(group): 
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count)

# Create the child nodes recursively
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left) 
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)
    
#Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):  
            return predict(node['left'], row) 
        else: 
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)   
        else:
            return node['right']

def subsample(dset, ratio):
    n_samp = round(len(dset)*ratio)
    sample = list()
    while len(sample) < n_samp:
        index = randrange(len(dset))
        sample.append(dset[index])
    return sample 

In [4]:
dset = load_csv('dataset.csv',header=0)
for i in range(0, len(dset[0])-1):
    str_col_to_float(dset, i)
str_col_to_int(dset, len(dset[0])-1)


def bagging_pred(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key = predictions.count)


def random_forest(train_data, test_data, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
            sample = subsample(train_data, sample_size)
            tree = build_tree(sample, max_depth, min_size, n_features)
            trees.append(tree)
    # Bag the results of all trees
    predictions = [bagging_pred(trees, row) for row in test_data]
    return(predictions)

seed(1)
n_folds = 3 
max_depth = 10 
min_size = 1 
sample_size = 1.0 
n_features = int(sqrt(len(dset[0])-1))
n_trees = [1]

for n_tree in n_trees:
    scores = eval_algorithm(dset, random_forest, n_folds, max_depth, min_size, sample_size, n_tree, n_features)

In [5]:
accuracy=(sum(scores) / float(len(scores)))
print("RF",accuracy)

RF 88.96094538117883


In [6]:
def ac_pred(dset, algorithm, k, *args):
    folds = cross_validation_split(dset, k)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_cpy = list(row)
            test_set.append(row_cpy)
            row_cpy[-1] = None 
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
    return actual,predicted

In [7]:
actual,predicted = ac_pred(dset, random_forest,n_folds, max_depth, min_size, sample_size, n_tree, n_features)

In [13]:
print('Precision:', metrics.precision_score(actual, predicted,average="weighted")*100)
print('Recall:', metrics.recall_score(actual, predicted,average="weighted")*100)
print('F1-Score:', metrics.f1_score(actual, predicted,average="weighted")*100)

Precision: 80.24689488854982
Recall: 89.58063121487247
F1-Score: 84.65727155175178


In [9]:
ACCURACY = metrics.accuracy_score(actual, predicted)*100
PRECISION = metrics.precision_score(actual, predicted,average="weighted")*100
RECALL = metrics.recall_score(actual, predicted,average="weighted")*100
F1_SCORE = metrics.recall_score(actual, predicted,average="weighted")*100

In [10]:
data=pd.DataFrame([['Random Forest',ACCURACY,PRECISION,RECALL,F1_SCORE]],columns=['Model','ACCURACY','PRECISION','RECALL','F1-SCORE'])
data.to_csv('scores_rf.csv',index=False)

In [11]:
data

Model   ACCURACY  PRECISION     RECALL   F1-SCORE
0  Random Forest  89.580631  80.246895  89.580631  89.580631